In [1]:
import json
from datetime import datetime, timedelta

In [2]:
with open('./records/2022-01-17.json') as timesheet_file:
    timesheet = json.load(timesheet_file)

In [3]:
# Need to validate dates in the file, making sure logged dates are after the Monday
# if we have a record 2022-01-17 there should be no dates before then, such as 2022-01-15

In [4]:
# Add an entry to work...
start = "2022-03-17T16:30"
end = "2022-03-17T18:00"
work_event = {"start": start, "end": end}

timesheet["work"].append(work_event)
print(timesheet["work"])

[{'start': '2022-01-17T08:15', 'end': '2022-01-17T12:00'}, {'start': '2022-01-17T13:00', 'end': '2022-01-17T16:30'}, {'start': '2022-02-18T08:15', 'end': '2022-02-18T12:00'}, {'start': '2022-02-18T12:30', 'end': '2022-02-18T16:00'}, {'start': '2022-03-19T08:30', 'end': '2022-03-19T12:00'}, {'start': '2022-03-20T12:30', 'end': '2022-03-20T16:00'}, {'start': '2022-03-20T16:30', 'end': '2022-03-20T18:00'}, {'start': '2022-03-17T16:30', 'end': '2022-03-17T18:00'}]


In [5]:
# Completed.
def convert_timedelta(duration):
    days, seconds = duration.days, duration.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = (seconds % 60)
    return hours, minutes, seconds

In [6]:
# Completed.
def sum_hours(logged_periods):
    timedeltas = list()
    for period in logged_periods:
        start = datetime.fromisoformat(period["start"])
        end = datetime.fromisoformat(period["end"])
        timedeltas.append(end-start)
    return timedeltas

In [7]:
work = sum(sum_hours(timesheet["work"]), timedelta())
hours, minutes, seconds = convert_timedelta(work)
print(f"Work is {hours=}, {minutes=}")

flex = sum(sum_hours(timesheet["flex"]), timedelta())
hours, minutes, seconds = convert_timedelta(flex)
print(f"Flex is {hours=}, {minutes=}")

holiday = sum(sum_hours(timesheet["holiday"]), timedelta())
hours, minutes, seconds = convert_timedelta(holiday)
print(f"Holiday is {hours=}, {minutes=}")

sick = sum(sum_hours(timesheet["sick"]), timedelta())
hours, minutes, seconds = convert_timedelta(sick)
print(f"Sick is {hours=}, {minutes=}")

total = work + flex + holiday + sick
hours, minutes, seconds = convert_timedelta(total)
print(f"Total is {hours=}, {minutes=}")

Work is hours=24, minutes=30
Flex is hours=3, minutes=30
Holiday is hours=3, minutes=30
Sick is hours=4, minutes=0
Total is hours=35, minutes=30


In [8]:
zero = timedelta(hours=0)

# Hours left in working week...
weekly_hours = timesheet["standard_working_hours_per_week"]["hours"]
weekly_minutes = timesheet["standard_working_hours_per_week"]["minutes"]
standard_weekly_hours = timedelta(hours=weekly_hours, minutes=weekly_minutes)

hours_left = abs(standard_weekly_hours - total)
hours, minutes, seconds = convert_timedelta(hours_left)

# can get rid of all this with timedelta
if hours_left >= zero:
    flextime = timedelta(hours=timesheet["flextime_balance"]["hours"], minutes=timesheet["flextime_balance"]["minutes"])
    flextime_increase = flextime + abs(hours_left)
    flex_hours, flex_minutes, flex_seconds = convert_timedelta(flextime_increase)
    print(f"You have worked more than your required hours! Your flextime will increase by {hours=}, {minutes=} to {flex_hours=}, {flex_minutes=}")
else:
    print(f"Hours left in working week {hours=}, {minutes=}")

You have worked more than your required hours! Your flextime will increase by hours=1, minutes=15 to flex_hours=4, flex_minutes=28


In [9]:
flextime = timedelta(hours=timesheet["flextime_balance"]["hours"], minutes=timesheet["flextime_balance"]["minutes"])

zero = timedelta(hours=0)

# Can get rid of this if-else with proper use of timedelta
if timesheet["flextime_balance"]["balance"].lower() == "credit":    
    hours, minutes, seconds = convert_timedelta(flextime)
    print(f"Your flex credit is currently {hours=}, {minutes=}.")

    # None of this makes sense... I think you need an if-statement to check if total is +/-
    flextime_decrease = standard_weekly_hours - total + flextime
    hours, minutes, seconds = convert_timedelta(flextime_decrease)    
    print(f"If you stop working now, this will increase to {hours=}, {minutes=}.")
    
    flextime_decrease = flextime - (standard_weekly_hours - total)
    hours, minutes, seconds = convert_timedelta(flextime_decrease) 
    print(f"If using up your flex credit then this will drop to {hours=}, {minutes=}.")
elif timesheet["flextime_balance"]["balance"].lower() == "debit":
    hours, minutes, seconds = convert_timedelta(flextime)
    print(f"Your flex debit is currently {hours=}, {minutes=}.")

    flextime_increase = standard_weekly_hours - total + flextime
    hours, minutes, seconds = convert_timedelta(flextime_increase)    
    print(f"If you stop working now, this will increase to {hours=}, {minutes=}.")

Your flex credit is currently hours=3, minutes=13.
If you stop working now, this will increase to hours=4, minutes=28.
If using up your flex credit then this will drop to hours=1, minutes=58.


In [10]:
with open('./records/2022-01-17.json', 'w') as timesheet_file:
    timesheet_json = json.dumps(timesheet, indent=4)
    timesheet_file.write(timesheet_json)